In [1]:
import pandas as pd
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

train_data = pd.read_csv("train_features.csv")
train_metadata = pd.read_csv("train_metadata.csv")
ana = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(ana)
ana.plot_data()


MachineLearningAnalyses 
  variables: 22 rows, 33 columns
  metadata: 22 rows, 14 columns



In [2]:
import numpy as np
from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn

iso = MachineLearningMethodIsolationForestSklearn(random_state = 30, contamination = 0.1)

#fits the model and upgrades analyses class to IsolationForestAnalyses
ana = iso.run(ana)
print(ana.__class__)

ana.plot_scores()

<class 'src.StreamPort.machine_learning.analyses.IsolationForestAnalyses'>


In [3]:
test_data = pd.read_csv("test_features.csv")
test_metadata = pd.read_csv("test_metadata.csv")
ana.predict(test_data, test_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)
ana.plot_scores()


   outlier     score
0     True -0.175769
1     True -0.196851
2     True -0.162131
3     True -0.182931


In [4]:
ana.add_prediction(add_outliers = False)
ana.plot_scores()

In [5]:
test_2_data = pd.read_csv("test_2_features.csv")
test_2_metadata = pd.read_csv("test_2_metadata.csv")
ana.predict(test_2_data, test_2_metadata)
outliers_test_2 = ana.test_prediction_outliers()
print(outliers_test_2)
ana.plot_scores()

   outlier     score
0    False  0.042166
1    False -0.019996
2    False -0.073318
3    False -0.002089


In [6]:
ana.add_prediction(add_outliers = False)
ana.plot_scores()